### Python Cassandra connection example

To run this section, you will need to have Cassandra up and running. Then using cqlsh,
<pre>CREATE KEYSPACE users_keyspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'} AND durable_writes = true;</pre>

Then create a table
<pre>
CREATE TABLE users_keyspace.users (
    firstname text,
    lastname text,
    email text,
    "favorite color" text,
    PRIMARY KEY (firstname, lastname)
) WITH CLUSTERING ORDER BY (lastname ASC);</pre>

Now you can insert some data
<pre>
INSERT INTO users_keyspace.users (
  firstname,
  lastname,
  email,
  "favorite color"
) VALUES (
  'Arthur',
  'Pendragon',
  'king.arthur@gmail.com',
  'red'
);

INSERT INTO users_keyspace.users (
  firstname,
  lastname,
  email,
  "favorite color"
) VALUES (
  'Sir',
  'Galahad',
  'sir.galahad@gmail.com',
  'blue'
);
</pre>

In [7]:
from cassandra.cluster import Cluster

In [15]:
cluster = Cluster(['127.0.0.1'], port=2042)

session = cluster.connect()

In [22]:
session.set_keyspace('users_keyspace')

In [23]:
rows = session.execute('SELECT firstname, lastname, email FROM users')
for user_row in rows:
    print(user_row.firstname, user_row.lastname, user_row.email)

AttributeError: 'str' object has no attribute 'firstname'

To do the next part, you will need to download the movies database from
https://grouplens.org/datasets/movielens/
Just download ml-latest-small.zip. It is in the "recommended for education and development" section

Then use cqlsh to load in the data.
First create a demo keyspace
<pre>create KEYSPACE demo WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};</pre>

Then create the movies table
<pre>create table demo.movies (movieId int primary key, title text, genres text);</pre>

Then load the data
<pre>COPY demo.movies from '/data/ml-latest-small/movies.csv' WITH HEADER = true;</pre>

This is assuming you have /data mounted as a VOLUME in your docker image to a local directory where you have unzipped the movie lens dataset

In [27]:
import pandas as pd 

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

In [25]:
session.row_factory = pandas_factory
session.default_fetch_size = None

In [30]:
query = "SELECT * from demo.movies limit 10"
# query
rslt = session.execute(query, timeout=None)
df = rslt._current_rows
df

InvalidRequest: Error from server: code=2200 [Invalid query] message="table salaires does not exist"

<pre>create KEYSPACE baseball WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};</pre>

<pre>create table baseball.salaries (playerID int primary key, year int, team text, name text, salary int);</pre>

<pre>COPY baseball.salaries from '/data/mlbSalaries.csv' WITH HEADER = true;</pre>

This is assuming you have /data mounted as a VOLUME in your docker image to a local directory where you have unzipped the movie lens dataset

In [41]:
query = "SELECT * from baseball.salaries limit 10"
# query
rslt = session.execute(query, timeout=None)
df = rslt._current_rows
df.groupby(['team']).agg({'salary': ['mean']}).reset_index()

,team,salary
,,mean
0,arizona-diamondbacks,3.682283e+05
1,chicago-cubs,7.405000e+06
2,cleveland-indians,3.097670e+05
3,kansas-city-royals,5.937000e+05
4,minnesota-twins,4.629629e+06
5,oakland-athletics,2.550000e+06
6,toronto-blue-jays,6.000000e+05
7,washington-nationals,3.000000e+06


In [53]:
query = "SELECT * from baseball.salaries where playerID=2342"
# query
rslt = session.execute(query, timeout=None)
df = rslt._current_rows
df

,playerid,salary,team
0,2342,8000000,san-francisco-giants
